In [ ]:
Met data van de Nederlandse Emissieautoriteit heb ik deze visualisatie 📈 gemaakt van jaarlijkse CO2 uitstoot,



Let op wat er met de kolencentrales (zwart) gebeurt. 

Uiteindelijk willen we al deze balkjes dus naar 0 krijgen, dat wordt nog een hele klus! 💪



Tips en verbeteringen welkom. In mei komt data van 2021 beschikbaar.

#emissions #climateneutral

In [2]:
import pandas as pd
import numpy as np

def map_columns(df, newcol):
    emissie_mapping = pd.read_csv('emissie_mapping.csv', sep=';')
    mapping = {k:v for k,v in zip(emissie_mapping['From'], emissie_mapping[newcol])}
    return df.rename(columns=mapping)

file = 'Emissiecijfers+2013-2020.ods'

emissions = pd.read_excel(file, engine='odf')
emissions = emissions.T
emissions.columns = emissions.loc['Inrichtingsnaam',:]
emissions.fillna(0, inplace=True)
emissions.drop(0, axis=1,inplace=True)

In [72]:
emissions.replace('nan', 0, inplace=True)
emissions.replace('NaN', 0, inplace=True)
emissions.replace('-', 0, inplace=True)
filtered_emissions = emissions[4:]
cols = filtered_emissions.columns
for c in cols:
    filtered_emissions[c] = filtered_emissions[c].apply(pd.to_numeric, errors='coerce')
    
filtered_emissions.index = filtered_emissions.index.map(lambda x: int(x.replace(' ','')[-4:]))
filtered_emissions /= 1000
filtered_emissions = filtered_emissions.rename(columns={'Warmte Station Galileïstraat':'Warmte Station Galileistraat'})
filtered_emissions.shape

C:\Users\GEBRUI~1\AppData\Local\Temp/ipykernel_20308/3595173266.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_emissions[c] = filtered_emissions[c].apply(pd.to_numeric, errors='coerce')


(8, 489)

In [111]:
# Uniper centrale maasvlakte powerplant 3 is een kolencentrale, er worden sinds 2017 geen emissies meer gemeld. Ik vermoed dat dit komt doordat de emissies zijn samengevoegd onder Uniper Centrale Maasvlakte
filtered_emissions['Uniper Centrale Maasvlakte'] += filtered_emissions['Uniper Maasvlakte Powerplant 3']
filtered_emissions.drop('Uniper Maasvlakte Powerplant 3', axis=1, inplace=True)


In [218]:
n_bars = 20
period_length_sec = 3
fps = 30
import bar_chart_race as bcr
import matplotlib.pyplot as plt
# Set up a figure looking like bar_chart_race's default.
# fig, ax = plt.subplots(figsize=(4, 2.5), dpi=144)
# ax.set_facecolor('.8')
# ax.tick_params(labelsize=8, length=0)
# ax.grid(True, axis='x', color='white')
# ax.set_axisbelow(True)
# [spine.set_visible(False) for spine in ax.spines.values()]
# Setting background to a solid color fixes the problem.
# fig.patch.set_facecolor('white')
def race(df, filename='EmissiesNL', pause=0, width=4, colors='dark12'):
    bcr.bar_chart_race(
        df=df,
        filename=filename+'.mp4',
        orientation='h',
        sort='desc',
        n_bars=n_bars,
        fixed_order=False,
        fixed_max=False,
        steps_per_period=fps * period_length_sec,
        interpolate_period=False,
        end_period_pause=pause,
        period_label={'x': .99, 'y': .25, 'ha': 'right', 'va': 'center', 'size': 8},
        # period_fmt='%B %d, %Y',
        period_summary_func=lambda v, r: {'x': .99, 'y': .18,
                                        's': f'Top {n_bars}: {v.nlargest(n_bars).sum()//1000:.0f} Mton CO2',
                                        'ha': 'right', 'size': 8},
        period_template='Jaar: {x:.0f}', 
        
        period_length=period_length_sec * 1000,
        
        perpendicular_bar_func=None, 
        # colors='dark12',
        colors=colors,
        title={
            'label': f'Per {filename} (Kton CO2 jaarlijks)',
            'size': 8,
            'color': 'black',
            'loc': 'left',
            'pad': 8
        },
        
        bar_size=.95, 
        bar_textposition='inside',
        bar_texttemplate='{x:.0f}', 
        bar_label_font=6, 
        tick_label_font=6, 
        tick_template='{x:,.0f}',
        shared_fontdict=None, 
        scale='linear', 
        fig=None, 
        writer=None, 
        bar_kwargs={'alpha': .7},
        fig_kwargs={'figsize': (width, 4), 'dpi': 300},
        filter_column_colors=False
    )

In [219]:
renamed_emissions = map_columns(filtered_emissions, 'Installatie')
grouped_emissions = renamed_emissions.groupby(renamed_emissions.columns, axis=1).sum()

cols = grouped_emissions.iloc[0,:].T.sort_values(ascending=False).index
grouped_emissions = grouped_emissions[cols]
type_to_color = {'staal':'grey',
                 'chemie': 'blue',
                 'kolen':'black',
                 'raffinaderij':'yellow',
                 'gas':'white',
                 'kerncentrale':'red'}

order = []
colors = []
for r in range(len(grouped_emissions)):
    cols = grouped_emissions.iloc[r,:].T.sort_values(ascending=False).index[:n_bars]
    for c in cols:
        if c not in order:
            order.append(c)
color_mapping = pd.read_csv('colors.csv', sep=';')
mapping = {k:v for k,v in zip(color_mapping['Naam'], color_mapping['Type'])}

for c in grouped_emissions.columns:
    if c in order:
        print(c, mapping[c], type_to_color[mapping[c]])
        colors.append(type_to_color[mapping[c]])
    else:
        colors.append('purple')

Uniper Centrale Maasvlakte kolen black
Tata Steel IJmuiden bv staal grey
Amercentrale kolen black
Chemelot chemie blue
Shell Nederland Chemie B.V. Pernis raffinaderij yellow
Vattenfall Centrale Hemweg kolen black
Dow Benelux B.V. chemie blue
Vattenfall Velsen gas white
Yara Sluiskil B.V. chemie blue
ENGIE Energie Centrale Gelderland kolen black
Shell Nederland Chemie B.V. Moerdijk chemie blue
EPZ Conventional Operations kolen black
ESSO Raffinaderij Rotterdam raffinaderij yellow
Vattenfall IJmond gas white
BP Raffinaderij Rotterdam B.V. raffinaderij yellow
ENGIE Eemscentrale gas white
Zeeland Refinery N.V. raffinaderij yellow
ENGIE Maximacentrale gas white
Pergen VOF chemie blue
Sloe Centrale B.V. gas white
Vattenfall Centrale Diemen gas white
Vattenfall Eemsmond gas white
Energie Productie Clauscentrale gas white
Enecogen gas white
Power Plant Rotterdam B.V. gas white
RWE Eemshaven Centrale kolen black


In [220]:
race(grouped_emissions, 'installatie', pause=0, width=6, colors=colors)

In [221]:
# Now per company
renamed_emissions = map_columns(filtered_emissions, 'Bedrijf')
grouped_emissions = renamed_emissions.groupby(renamed_emissions.columns, axis=1).sum()



We make a few adjudstments where there are joint ventures

In [222]:
grouped_emissions['Total'] += round(grouped_emissions['Zeeland Refinery N.V.'] * 0.55)
grouped_emissions['LUKoil'] = round(grouped_emissions['Zeeland Refinery N.V.'] * 0.45)
grouped_emissions.drop('Zeeland Refinery N.V.', axis=1, inplace=True)


In [223]:
grouped_emissions['PZEM'] = round(grouped_emissions['Sloe Centrale B.V.'] * 0.50)
grouped_emissions['EDF'] = round(grouped_emissions['Sloe Centrale B.V.'] * 0.50)
grouped_emissions.drop('Sloe Centrale B.V.', axis=1, inplace=True)


In [224]:
grouped_emissions['Eneco'] += round(grouped_emissions['Enecogen'] * 0.50)
grouped_emissions['Castleton'] += round(grouped_emissions['Enecogen'] * 0.50)
grouped_emissions.drop('Enecogen', axis=1, inplace=True)
 

In [225]:
race(grouped_emissions, 'bedrijf', pause=0, width=4, colors=['red'])

For further analysis

In [226]:
total = grouped_emissions.sum(axis=0).sort_values(ascending=False)
annual = grouped_emissions.sum(axis=1)
last_year = grouped_emissions.iloc[-1,:].T.sort_values(ascending=False)

In [217]:
widthinstallatie = 1800
widthbedrijf = 1200
overlap = 100
total = widthinstallatie + widthbedrijf - overlap
height = 1200

In [179]:
# Concat the two output files 
# make sure output file is not in directory otherwise doesnt work from notebook
# nullsrc is output size
# the two scales are input sizes
# the overlay is where number 2 should start
f'ffmpeg -i installatie.mp4 -i bedrijf.mp4 -filter_complex " nullsrc=size={total}x{height} [base];[0:v] setpts=PTS-STARTPTS, scale={widthinstallatie}x{height} [upperleft]; [1:v] setpts=PTS-STARTPTS, scale={widthbedrijf}x{height} [upperright]; [base][upperleft] overlay=shortest=1 [tmp1]; [tmp1][upperright] overlay=x={widthinstallatie-overlap}" -c:v libx264 total.mp4'

'ffmpeg -i installatie.mp4 -i bedrijf.mp4 -filter_complex " nullsrc=size=2900x1200 [base];[0:v] setpts=PTS-STARTPTS, scale=1800x1200 [upperleft]; [1:v] setpts=PTS-STARTPTS, scale=1200x1200 [upperright]; [base][upperleft] overlay=shortest=1 [tmp1]; [tmp1][upperright] overlay=x=1700" -c:v libx264 total.mp4'